In [1]:
from tqdm.notebook import tqdm
import torch
import pandas as pd
import torch.nn as nn
import transformers
from utils.dataset import MyDataset
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings('ignore')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

2023-12-31 23:28:06.394803: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-31 23:28:06.438071: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 23:28:07.035212: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from utils.common_functions import get_emotions
emotions = get_emotions().values()
n_labels = len(emotions)
def get_labels():
    labels = list(emotions)
    return labels

In [3]:
# Loading datasets into DF
df = pd.read_csv("/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/emotion-data/goemotion_simplified.csv")

# Checking Shape
display(df.shape)

(43410, 33)

In [4]:
from utils.preprocessing import get_clean_dataset
df = get_clean_dataset(df)
df.head()

,Unnamed: 0.1,Unnamed: 0,text,labels,id,0,1,2,3,4,...,18,19,20,21,22,23,24,25,26,27
0,0,0,favourite food anything didnt have cook myself,[27],eebbqej,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,does himself everyone will think having laugh ...,[27],ed00q6i,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2,FUCK BAYLESS ISOING,[2],eezlygj,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,make feel threatened,[14],ed7ypvh,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,Dirty Southern Wankers,[3],ed0bdzj,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from utils.dataset import Dataset

def build_dataset(tokenizer_max_len, tokenizer):
    dataset = Dataset(df.text.tolist(), [], tokenizer, tokenizer_max_len)
    return dataset
    # return train_dataset, valid_dataset

def build_dataloader(dataset, batch_size):
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    return data_loader
    # return train_data_loader, valid_data_loader

In [6]:
from utils.bert_classifier import get_model_tokenizer, Tokenize, Classification
# Test sentiment prediction
# test_text = "I love you"
model, tokenizer = get_model_tokenizer(pred_type='trend_pred',
                                mode_pt_file='best_trend_bert-base-uncased-29_12_2023-20_28_54.pt', 
                                model_ckpt='bert-base-uncased', labels=28, is_freez=1, gpu='cuda')

# dict = Tokenize(text=comment, max_length=30, tokenizer=tokenizer)
# scores = Classification(dict=dict, model=model, gpu='cuda:0')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
from utils.test import test_fn

dataset = build_dataset(tokenizer_max_len=30, tokenizer=tokenizer)
data_loader = build_dataloader(dataset, batch_size=16)
preds = test_fn(data_loader=data_loader, model=model, device='cuda')

  0%|          | 0/2714 [00:00<?, ?it/s]

In [8]:
preds = torch.stack(preds)
preds = preds.cpu().detach().numpy()

In [9]:
# comment_ids = text_ids.tolist()
comments_texts = df.text.tolist()


In [10]:
labels = get_labels()
df1 = df[[str(i) for i in range(len(labels))]]

In [11]:
map_list = []
trends = ["approval","toxic","obscene", 'insult', "threat", "hate", "offensive", "neither"]
for i, t in enumerate(comments_texts):
    data_map = {}
    # data_map['id'] = id
    data_map['text'] = comments_texts[i]
    for j, label in enumerate(trends):
        data_map[label] = preds[i][j]
    map_list.append(data_map)

In [12]:
df_trend_emotion = pd.DataFrame(data=map_list)
df_trend_emotion.head(5)

,text,approval,toxic,obscene,insult,threat,hate,offensive,neither
0,favourite food anything didnt have cook myself,0.875728,0.051118,0.008415,0.010713,0.005039,0.017730,0.014372,0.016885
1,does himself everyone will think having laugh ...,0.486363,0.102783,0.065887,0.046521,0.007450,0.051170,0.234065,0.005762
2,FUCK BAYLESS ISOING,0.969134,0.003580,0.002072,0.002417,0.001481,0.004402,0.009700,0.007212
3,make feel threatened,0.904893,0.012691,0.004578,0.004405,0.002808,0.007799,0.011963,0.050864
4,Dirty Southern Wankers,0.084473,0.035871,0.011421,0.014819,0.005246,0.230698,0.468812,0.148660


In [13]:
df_new = pd.concat([df_trend_emotion, df1], axis=1)

In [14]:
df_new.head(20)

,text,approval,toxic,obscene,insult,threat,hate,offensive,neither,0,...,18,19,20,21,22,23,24,25,26,27
0,favourite food anything didnt have cook myself,0.875728,0.051118,0.008415,0.010713,0.005039,0.017730,0.014372,0.016885,0,...,0,0,0,0,0,0,0,0,0,1
1,does himself everyone will think having laugh ...,0.486363,0.102783,0.065887,0.046521,0.007450,0.051170,0.234065,0.005762,0,...,0,0,0,0,0,0,0,0,0,1
2,FUCK BAYLESS ISOING,0.969134,0.003580,0.002072,0.002417,0.001481,0.004402,0.009700,0.007212,0,...,0,0,0,0,0,0,0,0,0,0
3,make feel threatened,0.904893,0.012691,0.004578,0.004405,0.002808,0.007799,0.011963,0.050864,0,...,0,0,0,0,0,0,0,0,0,0
4,Dirty Southern Wankers,0.084473,0.035871,0.011421,0.014819,0.005246,0.230698,0.468812,0.148660,0,...,0,0,0,0,0,0,0,0,0,0
5,pEyToN iSnT gOoD eNoUgH hElP PlAyOfFs Dumbass ...,0.958869,0.005908,0.002072,0.003409,0.001963,0.004233,0.004741,0.018804,0,...,0,0,0,0,0,0,0,0,1,0
6,heard bombs That Thanks your reply until then ...,0.854598,0.062224,0.011527,0.010214,0.004908,0.015314,0.017984,0.023232,0,...,0,0,0,0,0,0,0,0,0,0
7,need more boards create more space NAME Then good,0.958377,0.006169,0.001563,0.003573,0.001680,0.007904,0.004847,0.015887,0,...,0,0,1,0,0,0,0,0,0,0
8,Damn youtube outrage drama super lucrative reddit,0.060409,0.835658,0.027973,0.048932,0.006320,0.008722,0.007054,0.004933,1,...,0,0,0,0,0,0,0,0,0,0
9,might linked trust factor your friend,0.809066,0.021520,0.011884,0.008244,0.006326,0.028148,0.039417,0.075394,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
from utils.common_functions import get_emotions_dict
dict = get_emotions_dict()
df_new = df_new.rename(columns=dict)

In [18]:
df_new.to_csv('/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/emotion-data/trend_emotion_mapping(updated-D1).csv')